In [8]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer
from qdrant_client.http import models

In [2]:
client = QdrantClient(path="db")

In [3]:
sentences=[ "I like to be in my house",
"“I enjoy staying home",
"the isotope 238u decays to 206pb"]

In [4]:
encoder=SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [9]:
client.create_collection(
    collection_name="test_db", 
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE,
    ),
)

True

In [12]:
client.upload_points(
    collection_name="test_db", 
    points=[
        models.PointStruct(
            id=i,
            vector=encoder.encode(s),
            payload={"id": i, "sentence": s}
        ) for i, s in enumerate(sentences)
    ],
    parallel=4
)

In [14]:
client.search(
    collection_name="test_db",
    query_vector=encoder.encode("Do you like your house?"),
    limit=4
    
)


[ScoredPoint(id=0, version=0, score=0.6388576030731201, payload={'id': 0, 'sentence': 'I like to be in my house'}, vector=None, shard_key=None),
 ScoredPoint(id=1, version=0, score=0.45474663376808167, payload={'id': 1, 'sentence': '“I enjoy staying home'}, vector=None, shard_key=None),
 ScoredPoint(id=2, version=0, score=0.028513900935649872, payload={'id': 2, 'sentence': 'the isotope 238u decays to 206pb'}, vector=None, shard_key=None)]